#### Imports

In [1]:
from __future__ import division

import os
from inspect import getsourcefile
import sys
import pdb

# add parent dir to PYTHONPATH
current_path = os.path.abspath(getsourcefile(lambda:0))
current_dir = os.path.dirname(current_path)
parent_dir = current_dir[:current_dir.rfind(os.path.sep)]
sys.path.insert(0, parent_dir)

from collections import defaultdict
from copy import deepcopy

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from tqdm import tqdm
from torch.autograd import Variable
from tensorboardX import SummaryWriter

from src.dataset.neural_dataset import TrainDataset, ValDataset
from torch.utils.data.dataloader import DataLoader as PytorchDataLoader
from src.utils import get_config, get_csv_folds
from src.dataset.idris_dataset import IdrisDataset
from src.dataset.h5like_interface import H5LikeFileInterface
from src.transforms import augment_color

from src.pytorch_zoo import linknet

#### model and loss

In [2]:
models = {
    'linknet34': linknet.LinkNet34
}

def dice_loss(preds, trues, weight=None, is_average=True):
    num = preds.size(0)
    preds = preds.view(num, -1)
    trues = trues.view(num, -1)
    if weight is not None:
        w = torch.autograd.Variable(weight).view(num, -1)
        preds = preds * w
        trues = trues * w
    intersection = (preds * trues).sum(1)
    scores = 2. * (intersection + 1) / (preds.sum(1) + trues.sum(1) + 1)

    if is_average:
        score = scores.sum()/num
        return torch.clamp(score, 0., 1.)
    else:
        return scores

def dice_clamp(preds, trues, is_average=True):
    preds = torch.round(preds)
    return dice_loss(preds, trues, is_average=is_average)

class DiceLoss(nn.Module):
    def __init__(self, size_average=True):
        super(DiceLoss, self).__init__()
        self.size_average = size_average

    def forward(self, input, target, weight=None):
        return 1-dice_loss(F.sigmoid(input), target, weight=weight, is_average=self.size_average)

class BCEDiceLoss(nn.Module):
    def __init__(self, size_average=True):
        super(BCEDiceLoss, self).__init__()
        self.size_average = size_average
        self.dice = DiceLoss(size_average=size_average)

    def forward(self, input, target, weight=None):
        return nn.modules.loss.BCEWithLogitsLoss(size_average=self.size_average, weight=weight)(input, target) + self.dice(input, target, weight=weight)


def adjust_lr(optimizer, epoch, init_lr=0.1, num_epochs_per_decay=10, lr_decay_factor=0.1):
    lr = init_lr * (lr_decay_factor ** (epoch // num_epochs_per_decay))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr


def cyclic_lr(optimizer, epoch, init_lr=1e-4, num_epochs_per_cycle=5, cycle_epochs_decay=2, lr_decay_factor=0.5):
    epoch_in_cycle = epoch % num_epochs_per_cycle
    lr = init_lr * (lr_decay_factor ** (epoch_in_cycle // cycle_epochs_decay))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

losses = {
    'bce_dice_loss': BCEDiceLoss
}

#### trainer

In [3]:
class PytorchTrain:
    def __init__(self, fold, config, metrics):
        logdir = os.path.join('..', 'logs', config.folder, 'fold{}'.format(fold))
        if not os.path.exists(logdir):
            os.makedirs(logdir)
        self.config = config
        self.fold = fold
        self.model = models[config.network](num_classes=1, num_channels=config.num_channels)
        if self.config.optimizer == 'adam':
            self.optimizer = optim.Adam(self.model.parameters(), lr=config.lr)
        else:
            self.optimizer = optim.RMSprop(self.model.parameters(), lr=config.lr)
        self.model = nn.DataParallel(self.model).cuda()
        self.criterion = losses[config.loss]().cuda()
        self.writer = SummaryWriter(logdir)
        self.metrics = metrics
        self.devices = os.getenv('CUDA_VISIBLE_DEVICES', '0')
        if os.name == 'nt':
            self.devices = ','.join(str(d + 5) for d in map(int, self.devices.split(',')))

        self.cache = None
        self.cached_loss = 0
        self.hard_mining = True

    def run_one_epoch(self, epoch, loader, training=True):
        avg_meter = defaultdict(float)

        if training:
            pbar = tqdm(enumerate(loader), total=len(loader), desc="Epoch {}".format(epoch), ncols=0)
        else:
            pbar = enumerate(loader)
        for i, data in pbar:
            meter = self.make_step(data, training)
            for k, val in meter.items():
                avg_meter[k] += val
                if training and self.hard_mining and k == 'loss':
                    loss = val.cpu().numpy()[0]
                    if self.cache is None or self.cached_loss < loss:
                        self.cached_loss = loss
                        self.cache = deepcopy(data)

            if training:
                if self.hard_mining and i % 10 == 0 and self.cache is not None:
                    self.make_step(self.cache, training)
                    self.cache = None
                    self.cached_loss = 0
                pbar.set_postfix(**{k: "{:.5f}".format(v.cpu().numpy()[0] / (i + 1)) for k, v in avg_meter.items()})
        return {k: v.cpu().numpy()[0] / len(loader) for k, v in avg_meter.items()}

    def make_step(self, data, training):
        ret = {}
        images = data['image']
        ytrues = data['mask']
        if training:
            images = Variable(images.cuda(async=True))
            ytrues = Variable(ytrues.cuda(async=True))
        else:
            images = Variable(images.cuda(async=True), volatile=True)
            ytrues = Variable(ytrues.cuda(async=True), volatile=True)

        if training:
            self.optimizer.zero_grad()

        ypreds = self.model(images)
        loss = self.criterion(ypreds, ytrues)

        ret['loss'] = loss.data
        for name, func in self.metrics:
            acc = func(F.sigmoid(ypreds)[:, :, 6:-6, 6:-6].contiguous(), ytrues[:, :, 6:-6, 6:-6].contiguous())
            ret[name] = acc.data
        if training:
            loss.backward()
            torch.nn.utils.clip_grad_norm(self.model.parameters(), 1.)
            self.optimizer.step()

        return ret

    def fit(self, train_loader, val_loader):
#         save_path = os.path.join(self.config.models_dir, 'alxndrkalinin', self.config.folder)
        save_path = os.path.join(self.config.models_dir, self.config.folder)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        best_epoch = -1
        best_loss = float('inf')
        for epoch in range(self.config.nb_epoch):
            if epoch == self.config.cycle_start_epoch:
                print("Starting cyclic lr")
                self.optimizer = optim.Adam(self.model.parameters(), lr=self.config.lr)
            if epoch >= self.config.cycle_start_epoch:
                lr = cyclic_lr(self.optimizer, epoch - self.config.cycle_start_epoch, init_lr=self.config.lr, num_epochs_per_cycle=5, cycle_epochs_decay=2, lr_decay_factor=0.1)
            else:
                lr = adjust_lr(self.optimizer, epoch, init_lr=self.config.lr, num_epochs_per_decay=12)

            self.model.train()
            train_metrics = self.run_one_epoch(epoch, train_loader)
            self.model.eval()
            val_metrics = self.run_one_epoch(epoch, val_loader, training=False)
            print(" | ".join("{}: {:.5f}".format(k, float(v)) for k, v in val_metrics.items()))

            for k, v in train_metrics.items():
                self.writer.add_scalar('train/{}'.format(k), float(v), global_step=epoch)

            for k, v in val_metrics.items():
                self.writer.add_scalar('val/{}'.format(k), float(v), global_step=epoch)

            self.writer.add_scalar('lr', float(lr), global_step=epoch)

            loss = float(val_metrics['loss'])
            if loss < best_loss:
                best_loss = loss
                best_epoch = epoch
                torch.save(deepcopy(self.model), os.path.join(save_path, 'fold{}_best.pth'.format(self.fold)))
        print('Finished {} fold: {} with best loss {:.5f} on epoch {}\n'.format(self.config.folder, self.fold, best_loss, best_epoch))
        self.writer.close()

In [4]:
def train(ds, folds, config, num_workers=0, transforms=None, skip_folds=None):
    if not os.path.exists(os.path.join('..', 'weights')):
        os.makedirs(os.path.join('..', 'weights'))
    if not os.path.exists(os.path.join('..', 'logs')):
        os.makedirs(os.path.join('..', 'logs'))

    for fold, (train_idx, val_idx) in enumerate(folds):
        if skip_folds and fold in skip_folds:
            continue

        tr = TrainDataset(ds, train_idx, config, transform=transforms)
        val = ValDataset(ds, val_idx, config, transform=None)
        train_loader = PytorchDataLoader(tr,
                                         batch_size=config.batch_size,
                                         shuffle=True,
                                         drop_last=True,
                                         num_workers=num_workers,
                                         pin_memory=True)
        
        val_loader = PytorchDataLoader(val,
                                       batch_size=config.batch_size,
                                       shuffle=False,
                                       drop_last=False,
                                       num_workers=num_workers,
                                       pin_memory=True)
        trainer = PytorchTrain(fold=fold,
                               config=config,
                               metrics=[('soft dice', dice_loss),
                                        ('hard dice', dice_clamp),
                                        ('bce', nn.modules.loss.BCELoss())])
        trainer.fit(train_loader, val_loader)
        trainer.writer.close()


def train_config(config_path, folds_csv, n_folds=3, num_workers=0):
    config = get_config(config_path)

    root = config.dataset_path
    image_folder_name = 'retinopathy'

    ds = H5LikeFileInterface(IdrisDataset(root,
                                            config.img_rows, config.img_cols,
                                            image_folder_name=image_folder_name,
                                            apply_clahe=config.use_clahe))

    folds = get_csv_folds(folds_csv, os.listdir(os.path.join(root, image_folder_name)), n_folds)

    if num_workers == 0:
        if os.name != 'nt':
            num_workers = 5
#     num_workers = 0

    skip_folds = [i for i in range(n_folds) if config.fold is not None and i != int(config.fold)]
    print('skipping folds: ', skip_folds)
    train(ds, folds, config, num_workers=num_workers, transforms=augment_color, skip_folds=skip_folds)



#### training

In [5]:
train_config('../configs/idris_1.json', '../folds_5.csv', n_folds=5, num_workers=24)

{u'target_cols': 800, u'save_images': True, u'dbg': False, u'train_pad': 0, u'use_clahe': False, u'network': u'linknet34', u'use_crop': True, u'nb_epoch': 100, u'epoch_size': 1, 'models_dir': u'models', u'lr': 0.01, u'cycle_start_epoch': 50, u'folder': u'train', u'optimizer': u'adam', u'target_rows': 800, 'fold': None, u'batch_size': 16, u'test_pad': 0, u'use_resize': False, u'loss': u'bce_dice_loss', u'img_cols': 4288, u'predict_batch_size': 5, u'num_channels': 3, u'dataset_path': u'/home/user/data/idrid', u'img_rows': 2848}
('NAMES:', [u'IDRiD_28.jpg', u'IDRiD_44.jpg', u'IDRiD_23.jpg', u'IDRiD_32.jpg', u'IDRiD_11.jpg', u'IDRiD_51.jpg', u'IDRiD_54.jpg', u'IDRiD_31.jpg', u'IDRiD_14.jpg', u'IDRiD_10.jpg', u'IDRiD_30.jpg', u'IDRiD_03.jpg', u'IDRiD_04.jpg', u'IDRiD_43.jpg', u'IDRiD_40.jpg', u'IDRiD_19.jpg', u'IDRiD_50.jpg', u'IDRiD_48.jpg', u'IDRiD_49.jpg', u'IDRiD_41.jpg', u'IDRiD_01.jpg', u'IDRiD_45.jpg', u'IDRiD_15.jpg', u'IDRiD_09.jpg', u'IDRiD_38.jpg', u'IDRiD_52.jpg', u'IDRiD_27.jpg

Epoch 0: 100% 2/2 [00:12<00:00,  6.40s/it, bce=0.37539, hard dice=0.37823, loss=1.37299, soft dice=0.00286]


soft dice: 0.15611 | loss: 1.04363 | hard dice: 0.91372 | bce: 0.01564


Epoch 1: 100% 2/2 [00:07<00:00,  3.93s/it, bce=0.02306, hard dice=0.68962, loss=0.58945, soft dice=0.44487]


soft dice: 0.91372 | loss: 13.02560 | hard dice: 0.91372 | bce: 0.01599


...

soft dice: 0.71815 | loss: 0.32714 | hard dice: 0.71825 | bce: 0.02908


Epoch 98: 100% 2/2 [00:07<00:00,  3.64s/it, bce=0.01975, hard dice=0.82185, loss=0.19877, soft dice=0.82148]


soft dice: 0.71817 | loss: 0.32731 | hard dice: 0.71825 | bce: 0.02928


Epoch 99: 100% 2/2 [00:07<00:00,  3.96s/it, bce=0.03133, hard dice=0.62794, loss=0.40383, soft dice=0.62767]


soft dice: 0.71814 | loss: 0.32699 | hard dice: 0.71825 | bce: 0.02894
Finished train fold: 4 with best loss 0.32583 on epoch 90



In [7]:
train_config('../configs/idris_1.json', '../folds_3.csv', n_folds=3, num_workers=24)

{u'target_cols': 800, u'save_images': True, u'dbg': False, u'train_pad': 0, u'use_clahe': False, u'network': u'linknet34', u'use_crop': True, u'nb_epoch': 150, u'epoch_size': 1, 'models_dir': u'models', u'lr': 0.01, u'cycle_start_epoch': 30, u'folder': u'train', u'optimizer': u'adam', u'target_rows': 800, 'fold': None, u'batch_size': 16, u'test_pad': 0, u'use_resize': False, u'loss': u'bce_dice_loss', u'img_cols': 4288, u'predict_batch_size': 5, u'num_channels': 3, u'dataset_path': u'/home/user/data/idrid', u'img_rows': 2848}
('NAMES:', [u'IDRiD_28.jpg', u'IDRiD_44.jpg', u'IDRiD_23.jpg', u'IDRiD_32.jpg', u'IDRiD_11.jpg', u'IDRiD_51.jpg', u'IDRiD_54.jpg', u'IDRiD_31.jpg', u'IDRiD_14.jpg', u'IDRiD_10.jpg', u'IDRiD_30.jpg', u'IDRiD_03.jpg', u'IDRiD_04.jpg', u'IDRiD_43.jpg', u'IDRiD_40.jpg', u'IDRiD_19.jpg', u'IDRiD_50.jpg', u'IDRiD_48.jpg', u'IDRiD_49.jpg', u'IDRiD_41.jpg', u'IDRiD_01.jpg', u'IDRiD_45.jpg', u'IDRiD_15.jpg', u'IDRiD_09.jpg', u'IDRiD_38.jpg', u'IDRiD_52.jpg', u'IDRiD_27.jpg

Epoch 0: 100% 2/2 [00:07<00:00,  3.71s/it, bce=0.40810, hard dice=0.50312, loss=1.40523, soft dice=0.00377]


soft dice: 0.78332 | loss: 4.98604 | hard dice: 0.78332 | bce: 0.03120


Epoch 1: 100% 2/2 [00:07<00:00,  3.82s/it, bce=0.02928, hard dice=0.69058, loss=0.85528, soft dice=0.25132]


soft dice: 0.78332 | loss: 5.81757 | hard dice: 0.78332 | bce: 0.03120


...

soft dice: 0.69679 | loss: 0.36290 | hard dice: 0.69679 | bce: 0.03642


Epoch 148: 100% 2/2 [00:07<00:00,  3.75s/it, bce=0.03216, hard dice=0.56474, loss=0.47453, soft dice=0.56474]


soft dice: 0.69679 | loss: 0.36123 | hard dice: 0.69679 | bce: 0.03642


Epoch 149: 100% 2/2 [00:08<00:00,  4.01s/it, bce=0.02922, hard dice=0.68962, loss=0.34352, soft dice=0.68962]


soft dice: 0.69679 | loss: 0.36031 | hard dice: 0.69679 | bce: 0.03642
Finished train fold: 2 with best loss 0.34187 on epoch 110



In [8]:
# with CLAHE
train_config('../configs/idris_1.json', '../folds_5.csv', n_folds=5, num_workers=24)

{u'target_cols': 800, u'save_images': True, u'dbg': False, u'train_pad': 0, u'use_clahe': True, u'network': u'linknet34', u'use_crop': True, u'nb_epoch': 150, u'epoch_size': 1, 'models_dir': u'models', u'lr': 0.01, u'cycle_start_epoch': 30, u'folder': u'train', u'optimizer': u'adam', u'target_rows': 800, 'fold': None, u'batch_size': 16, u'test_pad': 0, u'use_resize': False, u'loss': u'bce_dice_loss', u'img_cols': 4288, u'predict_batch_size': 5, u'num_channels': 3, u'dataset_path': u'/home/user/data/idrid', u'img_rows': 2848}
('NAMES:', [u'IDRiD_28.jpg', u'IDRiD_44.jpg', u'IDRiD_23.jpg', u'IDRiD_32.jpg', u'IDRiD_11.jpg', u'IDRiD_51.jpg', u'IDRiD_54.jpg', u'IDRiD_31.jpg', u'IDRiD_14.jpg', u'IDRiD_10.jpg', u'IDRiD_30.jpg', u'IDRiD_03.jpg', u'IDRiD_04.jpg', u'IDRiD_43.jpg', u'IDRiD_40.jpg', u'IDRiD_19.jpg', u'IDRiD_50.jpg', u'IDRiD_48.jpg', u'IDRiD_49.jpg', u'IDRiD_41.jpg', u'IDRiD_01.jpg', u'IDRiD_45.jpg', u'IDRiD_15.jpg', u'IDRiD_09.jpg', u'IDRiD_38.jpg', u'IDRiD_52.jpg', u'IDRiD_27.jpg'

Epoch 0: 100% 2/2 [00:08<00:00,  4.11s/it, bce=0.36414, hard dice=0.25427, loss=1.36081, soft dice=0.00358]


soft dice: 0.00137 | loss: 1.01507 | hard dice: 0.91372 | bce: 0.01357


Epoch 1: 100% 2/2 [00:08<00:00,  4.26s/it, bce=0.03940, hard dice=0.37878, loss=0.91593, soft dice=0.14343]


soft dice: 0.91372 | loss: 57.23694 | hard dice: 0.91372 | bce: 0.01599


...

soft dice: 0.71819 | loss: 0.32788 | hard dice: 0.71825 | bce: 0.02992


Epoch 148: 100% 2/2 [00:08<00:00,  4.32s/it, bce=0.01669, hard dice=0.87777, loss=0.14110, soft dice=0.87752]


soft dice: 0.71816 | loss: 0.32716 | hard dice: 0.71825 | bce: 0.02919


Epoch 149: 100% 2/2 [00:08<00:00,  4.29s/it, bce=0.01964, hard dice=0.81572, loss=0.20612, soft dice=0.81524]


soft dice: 0.71814 | loss: 0.32677 | hard dice: 0.71825 | bce: 0.02878
Finished train fold: 4 with best loss 0.32436 on epoch 135



In [9]:
train_config('../configs/idris_1.json', '../folds_3.csv', n_folds=3, num_workers=24)

{u'target_cols': 800, u'save_images': True, u'dbg': False, u'train_pad': 0, u'use_clahe': True, u'network': u'linknet34', u'use_crop': True, u'nb_epoch': 150, u'epoch_size': 1, 'models_dir': u'models', u'lr': 0.01, u'cycle_start_epoch': 30, u'folder': u'train', u'optimizer': u'adam', u'target_rows': 800, 'fold': None, u'batch_size': 16, u'test_pad': 0, u'use_resize': False, u'loss': u'bce_dice_loss', u'img_cols': 4288, u'predict_batch_size': 5, u'num_channels': 3, u'dataset_path': u'/home/user/data/idrid', u'img_rows': 2848}
('NAMES:', [u'IDRiD_28.jpg', u'IDRiD_44.jpg', u'IDRiD_23.jpg', u'IDRiD_32.jpg', u'IDRiD_11.jpg', u'IDRiD_51.jpg', u'IDRiD_54.jpg', u'IDRiD_31.jpg', u'IDRiD_14.jpg', u'IDRiD_10.jpg', u'IDRiD_30.jpg', u'IDRiD_03.jpg', u'IDRiD_04.jpg', u'IDRiD_43.jpg', u'IDRiD_40.jpg', u'IDRiD_19.jpg', u'IDRiD_50.jpg', u'IDRiD_48.jpg', u'IDRiD_49.jpg', u'IDRiD_41.jpg', u'IDRiD_01.jpg', u'IDRiD_45.jpg', u'IDRiD_15.jpg', u'IDRiD_09.jpg', u'IDRiD_38.jpg', u'IDRiD_52.jpg', u'IDRiD_27.jpg'

Epoch 0: 100% 2/2 [00:09<00:00,  4.61s/it, bce=0.38579, hard dice=0.06532, loss=1.38417, soft dice=0.00221]


soft dice: 0.07132 | loss: 1.13334 | hard dice: 0.78332 | bce: 0.03064


Epoch 1: 100% 2/2 [00:08<00:00,  4.22s/it, bce=0.02850, hard dice=0.87532, loss=0.52836, soft dice=0.50618]


soft dice: 0.78332 | loss: 3539.97888 | hard dice: 0.78332 | bce: 0.03120


...

soft dice: 0.69679 | loss: 0.35644 | hard dice: 0.69679 | bce: 0.03586


Epoch 148: 100% 2/2 [00:08<00:00,  4.33s/it, bce=0.03590, hard dice=0.87572, loss=0.27395, soft dice=0.87572]


soft dice: 0.69679 | loss: 0.35504 | hard dice: 0.69679 | bce: 0.03572


Epoch 149: 100% 2/2 [00:08<00:00,  4.50s/it, bce=0.04049, hard dice=0.75195, loss=0.29347, soft dice=0.75195]


soft dice: 0.69679 | loss: 0.35435 | hard dice: 0.69679 | bce: 0.03568
Finished train fold: 2 with best loss 0.34178 on epoch 115

